In [104]:
# %pip install fake_useragent
import  csv, fake_useragent, os, requests, time
from bs4 import BeautifulSoup
'https://www.showcast.de/models?anrede=Frau&location_radius=10&start=160'
url ='https://www.showcast.de/models/?anrede=Frau&location_radius=10'
base_url = 'https://www.showcast.de'
basedir = "showcast.de"
page_set = set()
main_models_dict = dict()
models_details = list()


In [110]:
pages = ['/models/?anrede=Frau&location_radius=10']

In [121]:
def crawl(pages):
    found = list()
    while len(page_set) < 50:
        for page in pages:
            print(page)
            if page not in found:
                soup = BeautifulSoup(requests.get(base_url+page).text, 'html5lib')
                pagination = soup.find('ul', class_='pagination').findAll('a')
                new_pages=[page['href'] for page in pagination if page.get('href')]
                page_set.update(new_pages)
                found.append(page)
        print('PAGE SET',len(page_set))
        pages = list(page_set)


def get_models(url):
    """GETS THE URLS AND NAMES OF ALL MODELS ON THE WEBSITE"""
    print(url)
    soup = BeautifulSoup(requests.get(base_url+url).content, 'html5lib')
    models = soup.find('ul',class_='listusers').findAll('a')
    model_names = [' '.join(name.img['alt'].split()[1:]) for name in models]
    model_links = [link['href'] for link in models]
    models_dict = dict(zip(model_names, model_links))
    main_models_dict.update(models_dict)

            
def parse(url):
    model_info = dict()
    soup3 = BeautifulSoup(requests.get(base_url+url).content, 'html5lib')
    values = [val.text.strip() for val in soup3.find('div', class_ = 'view_profile').findAll('dd')[3:]]
    translation = ["HEIGHT", "Dimensions" , "CONFECTION", "CUP SIZE", "HAIR", "EYES", "SHOES"]
    for i,val in enumerate(values):
        model_info[translation[i]] = val
    return model_info

def download_pics(pics):
    for url in pics:
        ua = fake_useragent.UserAgent()
        path = f"{basedir}/{k}/{url.split('/')[-1]}"
        print(path)
        with open(path,'wb') as f:
            f.write(requests.get(f"{base_url}{url}", headers={'User-Agent':ua.random}).content)
def list_to_csv(lst):
    with open(f"{basedir}/models_info.csv",'w') as f:
        c = csv.DictWriter(f, ["NAME","URL", "HEIGHT", "Dimensions" , "CONFECTION", "CUP SIZE", "HAIR", "EYES", "SHOES"])
        c.writeheader()
        for row in lst:
            c.writerow(row)

crawl(pages)

In [113]:
for link in page_set:
    get_models(link)
print(len(main_models_dict))

/models?anrede=Frau&location_radius=10&start=80
/models?anrede=Frau&location_radius=10&start=340
/models?anrede=Frau&location_radius=10&start=800
/models?anrede=Frau&location_radius=10&start=420
/models?anrede=Frau&location_radius=10&start=200
/models?anrede=Frau&location_radius=10&start=40
/models?anrede=Frau&location_radius=10&start=0
/models?anrede=Frau&location_radius=10&start=680
/models?anrede=Frau&location_radius=10&start=660
/models?anrede=Frau&location_radius=10&start=160
/models?anrede=Frau&location_radius=10&start=280
/models?anrede=Frau&location_radius=10&start=240
/models?anrede=Frau&location_radius=10&start=60
/models?anrede=Frau&location_radius=10&start=820
/models?anrede=Frau&location_radius=10&start=180
/models?anrede=Frau&location_radius=10&start=740
/models?anrede=Frau&location_radius=10&start=120
/models?anrede=Frau&location_radius=10&start=720
/models?anrede=Frau&location_radius=10&start=920
/models?anrede=Frau&location_radius=10&start=600
/models?anrede=Frau&locat

In [117]:
for k,v in main_models_dict.items():
    os.makedirs(f"{basedir}/{k}", exist_ok=True)
    ua = fake_useragent.UserAgent()
    print(k,v)
    soup1 = BeautifulSoup(requests.get(base_url+v, headers={'User-Agent':ua.random}).content,'html5lib')
    feat_dict = parse(v)
    pics = [pic['href'] for pic in soup1.find('div', class_= 'tab-content').findAll('a')]
    download_pics(pics)
    print(len(pics))
    feat_dict['NAME'] = k
    feat_dict['URL'] = base_url+v
    models_details.append(feat_dict)
    

Anna-Lea /models/anna-lea_14580
showcast.de/Anna-Lea/avatar14580_b07a5ffdb48b0667e717d3335a6c9356.jpg
showcast.de/Anna-Lea/pdf114580_b3852f98348157f7c215f9848b38dfbe.jpg
showcast.de/Anna-Lea/pdf214580_34169fec10461db95e9d8baba630736f.jpg
showcast.de/Anna-Lea/_ralph_wietek_1_20170503_2097439727.jpg
showcast.de/Anna-Lea/_sven_heller_1_20170503_1745073553.jpg
showcast.de/Anna-Lea/_sven_heller_1_20170503_1584628074.jpg
showcast.de/Anna-Lea/_bemoremedia_1_20170503_1580773950.jpg
showcast.de/Anna-Lea/_ralph_wietek_1_20170503_1014233471.jpg
showcast.de/Anna-Lea/_lena_faye_1_20170503_1915324132.jpg
showcast.de/Anna-Lea/book_alm_3_20170504_1516119127.jpg
showcast.de/Anna-Lea/book_alm_7_20170504_1533599188.jpg
showcast.de/Anna-Lea/book_alm_9_20170504_1977571259.jpg
showcast.de/Anna-Lea/book_alm_30_20170504_1775318198.jpg
showcast.de/Anna-Lea/book_alm_35_20170504_1722985189.jpg
showcast.de/Anna-Lea/5c7398ff8187d1.20229756.jpg
showcast.de/Anna-Lea/5c73990f0a2818.71013164.jpg
showcast.de/Anna-Lea/5

In [119]:
len(models_details)

1

In [122]:
list_to_csv(models_details)